## 감정 분류기 작동 실험

### Google Drive 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**필요 패키지 설치**

In [ ]:
!pip install -r drive/'My Drive'/ML/KDT/Final/conversationModel/requirements.txt

### 환경 세팅

**Path 추가**

In [ ]:
import sys
sys.path.append('drive/My Drive/ML/KDT/Final/')
sys.path.append('drive/My Drive/ML/KDT/Final/conversationModel/')

**Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import random

from model.kobert import KoBERTforSequenceClassification, kobert_input
from kobert_transformers import get_tokenizer

#from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### 모델 로드 및 실행

In [ ]:
if __name__ == "__main__":
  root_path='drive/My Drive/ML/KDT/Final'
  checkpoint_path = f"{root_path}/conversationModel/checkpoint"
  labelencoder_path = f"{checkpoint_path}/labelencoder.pkl"
  save_ckpt_path = f"{checkpoint_path}/kobert-emotion-text-classification.pth"

  ctx = "cuda" if torch.cuda.is_available() else "cpu"
  device = torch.device(ctx)

  # 저장한 LabelEncoder 불러오기
  labelencoder = joblib.load(labelencoder_path)

  # 저장한 Checkpoint 불러오기
  checkpoint = torch.load(save_ckpt_path, map_location=device)

  model = KoBERTforSequenceClassification()
  model.load_state_dict(checkpoint['model_state_dict'])

  model.eval()

  tokenizer = get_tokenizer()

  while 1:
    sent = input('\nQuestion: ') # 문장 입력
    data = kobert_input(tokenizer,sent, device, 512)

    output = model(**data)

    logit = output[0]
    softmax_logit = torch.softmax(logit,dim=-1)
    softmax_logit = softmax_logit.squeeze()

    max_index = torch.argmax(softmax_logit).item()
    max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

    category = labelencoder.inverse_transform([max_index])

    print(f'Answer: {category}, index: {max_index}, softmax_value: {max_index_value}')
    print('-' * 50)


Question: 이렇게 하면 잘 작동할까..?
Answer: ['혼란스러운'], index: 52, softmax_value: 0.03761377930641174
--------------------------------------------------
